In [13]:

import osmnx as ox
import folium
import pandas as pd
import numpy as np
import utils
import folium_utils

from IPython.display import display
pd.set_option('display.max_rows', None)

# 峠道
graph = ox.graph_from_point(center_point=(35.334560, 136.989900)
                                , network_type='drive'
                                , dist=5000
                                , simplify=True
                                , custom_filter='["highway"~"tertiary|secondary|primary|trunk"]')

# グラフデータをGeoDataFrameに変換  
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# エッジの長さから評価する。
lower_bound_meter = 200
upper_bound_meter = 2000
gdf_edges['length_evaluation'] = np.where(gdf_edges['length'] <= lower_bound_meter, 0,
                                   np.where(gdf_edges['length'] >= upper_bound_meter, 1,
                                            (gdf_edges['length'] - lower_bound_meter) / (upper_bound_meter - lower_bound_meter)))

# エッジからジオメトリーのポイント数を取得する(ポイントが多い = Rがついている道?, エッジが重なっている所もポイント数が多くなるので注意(後々で別の評価関数を実装する予定))
gdf_edges['point_cnt'] = gdf_edges['geometry'].apply(lambda x: len(x.coords))

# エッジの距離とポイント数から曲線率?を求める。
gdf_edges['curvature'] = gdf_edges['point_cnt'] / gdf_edges['length']

# 3座標間の角度を求める
def calculate_angle_between_vectors(A, B, C):
    vector_AB = np.array(B) - np.array(A)
    vector_BC = np.array(C) - np.array(B)
    
    dot_product = np.dot(vector_AB, vector_BC)
    norm_AB = np.linalg.norm(vector_AB)
    norm_BC = np.linalg.norm(vector_BC)
    
    cosine_theta = dot_product / (norm_AB * norm_BC)
    angle_rad = np.arccos(cosine_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

# 座標間の角度の変化の合計値を求める
gdf_edges['geometory_angle_total'] = gdf_edges['geometry'].apply(
    lambda x: sum([calculate_angle_between_vectors(x.coords[i-1], x.coords[i], x.coords[i+1]) for i in range(1, len(x.coords)-1)])
)

# 抽出対象のエッジにフラグを立てる.条件はcurvatureが0.025より上 かつ length_evaluationが0.5以上
gdf_edges['is_target'] = np.where((gdf_edges['curvature'] > 0.025) & (gdf_edges['length_evaluation'] > 0.5) & (gdf_edges['geometory_angle_total'] > 120), 1, 0)

G = ox.graph_from_gdfs(gdf_nodes, gdf_edges)

# 地図を表示する
map_osm = ox.plot_graph_folium(ox.utils_graph.get_largest_component(G, strongly=True), edge_width=2)

# # 中央にマーカーを表示
# folium_utils.add_marker(map_osm, 35.334560, 136.989900, "st", "green")

# is_targetが正のエッジにplotを立てて緯度と経度を表示する
for index, row in gdf_edges[gdf_edges['is_target'] == 1].iterrows():
    folium_utils.add_marker(map_osm,
                            row['geometry'].coords[0][1],
                            row['geometry'].coords[0][0],
                            f"{row['geometry'].coords[0][1]}, {row['geometry'].coords[0][0]}, {row['geometory_angle_total']}, {row['length']}",
                            "red")

# 評価値を元にエッジのスタイルを変える
map_osm.add_child(
    folium.features.GeoJson(
        gdf_edges[gdf_edges['is_target'] == 1].to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#ff0000',
            'fill_opacity': 0,
            'weight': 5,
        }
    )
)


/tmp/ipykernel_7413/1728692222.py:64: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  map_osm = ox.plot_graph_folium(ox.utils_graph.get_largest_component(G, strongly=True), edge_width=2)
